# **Lab 3 - Activity Recognition with Machine Learning**

This notebook implements a machine learning workflow to recognize different physical activities from Respeck sensor data. The dataset includes multiple 30-second recordings of various physical activities (e.g., ascending stairs, shuffle walking, sitting-standing) stored in separate CSV files for each activity.

You will then use the model you develop here and deploy it inside your Android app for live classification.

In this week, you will not have access to the full dataset as of yet. However, you can complete this lab by combining the data that you and your group mates have collected in Coursework 1 as proof-of-concept first for when you eventually receive the full dataset.


# Imports

In [35]:
# Importing libraries that will be used
import pandas as pd
import numpy as np
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report

# Reading Files
Reading files from your dataset

In [36]:
# Put in the path of your dataset here
# Trying with Respeck data only first
your_dataset_path = "./PDIoT2324/Respeck/"

This line uses the glob module to find all file paths that match a specified pattern. The 'glob.glob()' function returns a list of file paths that match the given pattern. `your_dataset_path` should be the directory where your dataset files are located.

The `*` is a wildcard character that matches any string of characters,  so this pattern retrieves all folders in the 'your_dataset_path' directory.

Below is just an example of what your dataset folder can look like. You should refer to the Coursework 3 instructions on what classes your model(s) are expected to be able to classify. Within your dataset directory, there should be subfolders, each representing a class of activity.

In [37]:
glob.glob(your_dataset_path + "*")

['./PDIoT2324/Respeck\\s100_respeck_ascending_breathingNormal.csv',
 './PDIoT2324/Respeck\\s100_respeck_descending_breathingNormal.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_breathingNormal.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_coughing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_hyperventilating.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_laughing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_singing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_talking.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_breathingNormal.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_coughing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_hyperventilating.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_laughing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_singing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_talking.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingRight_breathingNormal.csv',
 './PDIoT2324/Respeck\\s100_respeck_

## Activities and Social Signals Dictionary

In [125]:
# Define activities and social signals with corresponding labels
# Each key is the name of the physical activity, and the corresponding value is the numeric label
# These labels will be used as the target variable for classification.
activities_dict = {
    'ascending': 0,
    'shuffleWalking': 1,
    'sittingStanding': 2,
    'miscMovement': 3,
    'normalWalking': 4,
    'lyingBack': 5,
    'lyingLeft': 6,
    'lyingRight': 7,
    'lyingStomach': 8,
    'descending': 9,
    'running': 10
}

social_signals_dict = {
    'breathingNormal': 0,
    'coughing': 1,
    'hyperventilating': 2,
    'other': 3
}

# Functions

## Load list of files in an activity folder

In [126]:
# Will need to do this for the Daily Activities folder and the Respiratory folder
def load_files_from_folder(folder_path):
    """
    Load all CSV files from a folder, extract activity and social signal information,
    and return a list of file paths along with combined labels.

    Parameters:
    folder_path (str): The path to the folder containing CSV files.

    Returns:
    tuple: A tuple containing:
        - list: A list of file paths for all CSV files in the folder.
        - list: A list of combined labels (activity and social signal) for each file.
    """

    # Initialise an empty list to store the full file paths of the CSV files
    file_paths = []

    # Initialse an empty dictionary to store the filenames and the activity and social signal labels
    file_info = {}

    # Initialize an empty list to store the combined labels of the activity and social signal to be used in stratified split of data
    # Stratified split of data ensures that each activity-social signal combination is proportionally represented in the training and testing data
    combined_labels = []

    # Loop through all the files in the given folder
    for file_name in os.listdir(folder_path):
        
        # Check if the file has a .csv extension (ignores other files)
        if file_name.endswith('.csv'):

            # Inialise an empty dictionary for each file to store the activity and social signal labels
            file_info[file_name] = {}

            # Split the file name by underscores to extract activity and social signal information
            parts = file_name.split("_")
            
            # Extract the activity from the file name
            activity = parts[2]

            # Add activity label to the file_info dictionary
            if activity == "sitting" or activity == "standing":
                file_info[file_name]['activity_label'] = activities_dict["sittingStanding"]
            else:
                file_info[file_name]['activity_label'] = activities_dict[activity]
            
            # Extract the social signal from the file name, without the .csv extension
            social_signal = parts[3].split(".")[0]

            # Add social signal label to the file_info dictionary
            if social_signal == "laughing" or social_signal == "eating" or social_signal == "talking" or social_signal == "singing":
                file_info[file_name]['social_signal_label'] = social_signals_dict["other"]
            else:
                file_info[file_name]['social_signal_label'] = social_signals_dict[social_signal]
            
            # Combine the activity and social signal to create a unique label
            combined_label = activity + "_" + social_signal

            # Append the combined label to the combined_labels list
            combined_labels.append(combined_label)

            # Construct the full file path by joining the folder path and the file name'
            full_file_path = os.path.join(folder_path, file_name)

            # Append the full file path to the file_paths list
            file_paths.append(full_file_path)

    # Return the complete list of CSV file paths
    return file_paths, combined_labels, file_info

## Train and test set split from list of files

In [127]:
# Will need to split files for the Daily Activities folder and the Respiratory folder
def split_files(file_list, combined_labels, test_size=0.2):
    """
    Split the list of file paths into training and testing sets.

    Parameters:
    file_paths (list): A list of file paths for all CSV files.
    combined_labels (list): A list of combined labels (activity and social signal) for each file.
    test_size (float, optional): The proportion of the dataset to include in the test split. Default is 0.2.

    Returns:
    tuple: A tuple containing:
        - list: Training file paths.
        - list: Testing file paths.
    """

    # Split the file list into training and test sets using train_test_split from scikit-learn
    # test_size defines the proportion of the data to use as the test set (default is 20%)
    # shuffle=True ensures that the files are shuffled randomly before splitting
    train_files, test_files = train_test_split(file_list, test_size=test_size, stratify = combined_labels, shuffle=True, random_state=42)

    # Return the train and test file lists
    return train_files, test_files

## Sliding Window

In time series Activity Recognition, a sliding window is a commonly used technique to segment continuous sensor data (such as accelerometer readings) into smaller, fixed-length overlapping or non-overlapping time intervals, or windows. Each window contains a sequence of sensor measurements that represent a short period of time, and this segmented data is used to extract features or make predictions about the activity happening within that window.

### Key Concepts of a Sliding Window
1.   **Window Size:** This refers to the length of each segment or window, typically defined in terms of the number of time steps or the duration (e.g., 2 seconds). The window size should be chosen carefully to capture enough information about the activity without making the window too large.
2.   **Step Size:** The step size determines how far the window moves forward after each step. If the step size is smaller than the window size, the windows will overlap. For example, if the window size is 5 seconds and the step size is 2 seconds, there will be a 3-second overlap between consecutive windows. Overlapping windows provide more data for analysis and can help smooth out predictions by capturing transitional activities.
3.   **Non-Overlapping Windows:** If the step size is equal to the window size, the windows do not overlap. This method provides distinct segments of data but may miss transitional phases between activities.

### Why Sliding Windows for Activity Recognition?

* Segmentation of Continuous Data: Activity recognition systems work with continuous streams of sensor data, and the sliding window helps segment these into manageable pieces to classify activities within specific intervals.

* Context Capturing: Human activities are often complex and spread across time. By using a sliding window, you can capture context across a short duration, which may include transitions or small fluctuations in the activity (e.g., a person moving from sitting to standing).

* Feature Extraction: Within each window, features such as mean, variance, frequency domain features, etc., can be extracted to help classify the activity.

* Real-Time Recognition: In real-time systems, the sliding window allows for continuous monitoring and updating of predictions as new data arrives.



In [128]:
def load_and_apply_sliding_windows(file_path, window_size, step_size, file_info):
    """
    Load the data from each file, apply sliding windows, and return the windows and labels.

    Parameters:
    file_paths (list): List of file paths to CSV files. Each file contains sensor data.
    window_size (int): The size of each sliding window (number of time steps).
    step_size (int): The step size (stride) between consecutive windows.
    activity_label (int or str): The label for the activity corresponding to the file.
                                 This label will be assigned to each sliding window extracted from the data.
    social_signal_label (int or str): The label for the social signal corresponding to the file.
                                      This label will be assigned to each sliding window extracted from the data.

    Returns:
    tuple:
        - windows (numpy.ndarray): A 3D array of sliding windows, where each window has the shape
                                   (num_windows, window_size, num_features).
        - labels (numpy.ndarray): A 1D array of labels, where each label corresponds to a sliding window.
    """
    # Initialize lists to store sliding windows and their corresponding labels
    windows = []
    activity_labels = []
    social_signal_labels = []
    file_number = 0

    
    # Loop through each file in the provided file path
    for file in file_path:
        file_number += 1

        # Extract the activity and social signal labels from the file_info dictionary
        activity_label = file_info[os.path.basename(file)]['activity_label']
        social_signal_label = file_info[os.path.basename(file)]['social_signal_label']

        # Load the CSV file into a pandas DataFrame
        data = pd.read_csv(file)   


        # Select the columns containing the necessary sensor data (acceleration readings)
        # These columns might vary depending on your dataset's structure
        data = data[['accel_x', 'accel_y', 'accel_z']]
        
        # Convert the DataFrame into a numpy array for faster processing in the sliding window operation
        data = data.to_numpy()

        
        # Get the number of samples (rows) and features (columns) in the data
        num_samples, num_features = data.shape
        
        # Apply sliding windows to the data
        # The range function defines the start of each window, moving step_size increments at a time
        for i in range(0, num_samples - window_size + 1, step_size):
            # Extract a window of size 'window_size' from the current position 'i'
            window = data[i:i + window_size, :]

            # Append the window to the windows list
            windows.append(window)

            # Assign the activity label to the window and append it to the labels list
            activity_labels.append(activity_label)
            social_signal_labels.append(social_signal_label)

    # Convert the lists of windows and labels into numpy arrays for efficient numerical operations
    return np.array(windows), np.array(activity_labels), np.array(social_signal_labels) 

    
    

## Debugging

In [129]:
# Load files from folder
file_paths, combined_labels, file_info = load_files_from_folder(your_dataset_path)
print(f"Loaded {len(file_paths)} files")
print(f"The file info dictionary is {file_info}")

Loaded 4064 files
The file info dictionary is {'s100_respeck_ascending_breathingNormal.csv': {'activity_label': 0, 'social_signal_label': 0}, 's100_respeck_descending_breathingNormal.csv': {'activity_label': 9, 'social_signal_label': 0}, 's100_respeck_lyingBack_breathingNormal.csv': {'activity_label': 5, 'social_signal_label': 0}, 's100_respeck_lyingBack_coughing.csv': {'activity_label': 5, 'social_signal_label': 1}, 's100_respeck_lyingBack_hyperventilating.csv': {'activity_label': 5, 'social_signal_label': 2}, 's100_respeck_lyingBack_laughing.csv': {'activity_label': 5, 'social_signal_label': 3}, 's100_respeck_lyingBack_singing.csv': {'activity_label': 5, 'social_signal_label': 3}, 's100_respeck_lyingBack_talking.csv': {'activity_label': 5, 'social_signal_label': 3}, 's100_respeck_lyingLeft_breathingNormal.csv': {'activity_label': 6, 'social_signal_label': 0}, 's100_respeck_lyingLeft_coughing.csv': {'activity_label': 6, 'social_signal_label': 1}, 's100_respeck_lyingLeft_hyperventilati

In [130]:
# Split files into train and test sets
train_files, test_files = split_files(file_paths, combined_labels)
print(f"Files have been split into {len(train_files)} training and {len(test_files)} testing files")

Files have been split into 3251 training and 813 testing files


In [131]:
# Limit the number of files for sliding window processing to speed up debugging
limited_train_files = train_files[:1000]
print(f"Processing {len(limited_train_files)} training files with sliding windows")

Processing 1000 training files with sliding windows


In [132]:
# Apply sliding windows
windows, activity_labels, social_signal_labels = load_and_apply_sliding_windows(train_files, 50, 50, file_info)
print(f"Generated {len(windows)} windows, {len(activity_labels)} activity labels and {len(social_signal_labels)} social signal labelsfrom {len(train_files)} files")
print(f"Activity labels: {activity_labels}, Social Signal labels = {social_signal_labels}")

Generated 48575 windows, 48575 activity labels and 48575 social signal labelsfrom 3251 files
Activity labels: [ 7  7  7 ... 10 10 10], Social Signal labels = [3 3 3 ... 0 0 0]


## Load and Split Train Test for Each Activity Folder

This function processes the sensor data for a specific activity, such as 'walking' or 'running', stored in its respective folder. It splits the data into training and testing sets, applies sliding windows, and labels the windows with the corresponding activity. This function can be used repeatedly for each activity to process and prepare data for training and evaluation.

In [137]:
def process_activity(dataset_path, window_size=50, step_size=50, test_size=0.2):
    """
    Processes an activity folder by loading the file list, splitting them into
    train and test sets, and applying sliding windows to the files.

    Args:
        activity (str): Name of the activity (folder name). This refers to the specific physical activity
                        like 'walking', 'running', etc.
        label (int): Numeric label corresponding to the activity, used for classification.
        dataset_path (str): Base path where the activity folders are located.
        window_size (int): Size of the sliding window, i.e., the number of time steps included in each window.
                           Default is 50.
        step_size (int): Step size for the sliding window, i.e., how far the window moves along the data.
                         Default is 50 (no overlap between windows).
        test_size (float): Proportion of files to use for testing. Default is 0.2, meaning 20% of files will
                           be allocated to the test set.

    Returns:
        tuple:
            - train_windows (numpy.ndarray): Sliding windows from the training files.
            - train_labels (numpy.ndarray): Corresponding labels for the training windows.
            - test_windows (numpy.ndarray): Sliding windows from the test files.
            - test_labels (numpy.ndarray): Corresponding labels for the test windows.
    """

    # Load all CSV file paths for the given activity from the folder
    file_paths, combined_labels, files_info = load_files_from_folder(dataset_path)

    # Split the file list into training and testing sets
    # train_files: files used for training
    # test_files: files used for testing
    train_files, test_files = split_files(file_paths, combined_labels, test_size=test_size)

    # Apply sliding windows to the training files
    # The function 'load_and_apply_sliding_windows' returns the sliding windows (segments) and their corresponding labels
    train_windows, train_activity_labels, train_social_signal_labels = load_and_apply_sliding_windows(train_files, window_size, step_size, file_info)

    # Apply sliding windows to the testing files
    test_windows, test_activity_labels, test_social_signal_labels = load_and_apply_sliding_windows(test_files, window_size, step_size, file_info)

    # Return the sliding windows and their labels for both training and testing sets
    return train_windows, train_activity_labels, train_social_signal_labels, test_windows, test_activity_labels, test_social_signal_labels

In [138]:
train_windows, train_activity_labels, train_social_signal_labels, test_windows, test_activity_labels, test_social_signal_labels = process_activity(your_dataset_path, test_size=0.2, window_size=50, step_size=50)
print(f"{len(train_windows)} train windows generated with {len(train_activity_labels)} activity labels and {len(train_social_signal_labels)} social signal labels")
print(f"{len(test_windows)} test windows generated with {len(test_activity_labels)} activity labels and {len(test_social_signal_labels)} social signal labels")
print(f"Activity labels: {train_activity_labels}, Social Signal labels = {train_social_signal_labels}")

48575 train windows generated with 48575 activity labels and 48575 social signal labels
12154 test windows generated with 12154 activity labels and 12154 social signal labels
Activity labels: [ 7  7  7 ... 10 10 10], Social Signal labels = [3 3 3 ... 0 0 0]


## 1D CNN Model

This function, `build_1d_cnn_model`, creates and compiles a 1D Convolutional Neural Network (CNN) for multi-class classification tasks.

### Function Overview

Input Parameters
* `input_shape`: Specifies the shape of the input data. It represents (timesteps, features), where timesteps refer to the length of the time series (e.g., 50 windows), and features represent the number of measurements in each time step (e.g., accelerometer readings).
* `num_classes`: The number of output classes for the classification problem. For example, if you're classifying six different activities, num_classes would be 6.

Returns
* The function returns a compiled 1D CNN model that is ready to be trained on your data.

<hr>

### Function Breakdown
1.   Model Initialization:
    * `model = Sequential()`: Initializes a Sequential model, which means layers will be stacked on top of each other in a linear fashion.
2. First Convolutional Layer:
    * `Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape)`
        * This is the first 1D convolutional layer
        * `filters=64`: The layer applies 64 filters (or kernels) over the input data.
        * `kernel_size=3`: Each filter will cover 3 timesteps at a time (a window of 3). Different from the sliding window size in the data preprocessing. Looks using a window of 3 timesteps at a time within the sliding window from data preprocessing.
        * `activation='relu'`: The Rectified Linear Unit (ReLU) activation function introduces non-linearity and helps the model learn complex patterns.
        * `input_shape=input_shape`: Specifies the shape of the input data.
    * `MaxPooling1D(pool_size=2)`: This pooling layer reduces the dimensionality of the data (i.e. reduces the size of the data along the time dimension) by taking the maximum value from each 2-timestep window (`pool_size=2`). So will keep the max value from the 2-timestep window. This helps reduce computational complexity and captures the most important features. Known as downsampling: Process to reduce the number of timesteps while keeping the number of features the same. 
3. Second Convolutional Layer:
    * `Conv1D(filters=128, kernel_size=3, activation='relu')`
        * This is the second convolutional layer, similar to the first, but with 128 filters, which allow the network to learn more complex features from the data.
        * `kernel_size=3` and activation='relu' function in the same way as the first Conv1D layer.
    * `MaxPooling1D(pool_size=2)`: Another pooling layer to downsample the output, further reducing the data’s dimensionality.
4. Flattening Layer:
    * `Flattening`: Converts the 2D output of the convolutional and pooling layers into a 1D vector. This is necessary because the next layer is fully connected, and it requires a 1D input.
5. Fully Connected Layer:
    * `Dense(128, activation='relu')`: This is a fully connected layer with 128 units/neurons. Each neuron is connected to every input from the flattened output. The ReLU activation function is used again to introduce non-linearity and help the model learn complex relationships.
6. Dropout Layer:
    * `Dropout(0.5)`: This layer randomly sets 50% of the neurons to zero during training to prevent overfitting. It helps the model generalize better to unseen data.
7. Output Layer:
    * `Dense(num_classes, activation='softmax')`: This is the output layer with num_classes neurons, one for each class in the classification problem. The softmax activation function ensures the output values represent probabilities that sum to 1, useful for multi-class classification.
8. Compiling the model
    * model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']):
        * Optimizer: 'adam': Adam is an optimization algorithm that adjusts the learning rate during training to improve performance.
        * Loss: 'categorical_crossentropy': This loss function is used for multi-class classification problems where the target variable is one-hot encoded (i.e., represented as a vector of 0s and 1s).
        * Metrics: ['accuracy']: The accuracy metric is used to evaluate the model’s performance during training and testing.


In [ ]:
def build_1d_cnn_model(input_shape, num_classes):
    """
    Builds and compiles a 1D CNN model for multi-class classification.

    Args:
        input_shape (tuple): The shape of the input data (timesteps, features).
        num_classes (int): The number of output classes.

    Returns:
        model (Sequential): Compiled 1D CNN model.
    """
    model = Sequential()

    # First Conv1D layer
    # You can try experimenting with different filters, kernel_size values and activiation functions
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))

    # Second Conv1D layer
    # You can try experimenting with different filters, kernel_size values and activiation functions
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    # Flatten the output from the convolutional layers
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(128, activation='relu'))

    # Dropout layer for regularization
    # You can try experimenting with different dropout rates
    model.add(Dropout(0.5))

    # Output layer with softmax for multi-class classification
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    #  Prints a detailed summary of the model, showing the layers, their output shapes, and the number of trainable parameters
    model.summary()

    return model

# Classification Pipeline

## Step 1: Prepare and Preprocess the Data

Now that each activity has been processed and stored in train_test_data, we need to combine the sliding windows and labels from all activities into unified arrays (one for training and one for testing) for model training.

In [139]:
# Generate the sliding windows along with activity and social signal labels for training and testing sets
X_train, y_train_activity, y_train_social_signal, X_test, y_test_activity, y_test_social_signal = process_activity(your_dataset_path, test_size=0.2, window_size=50, step_size=50)


# Explanation:
# - process_activity() function is used to generate sliding windows and labels for the training and testing sets.
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.


# Check that these training and testing sets generated by process activity above have the correct shape.

In [140]:
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

NameError: name 'y_train' is not defined

### One-Hot Encode Labels (for multi-class classification)
If you have more than two classes, you'll need to one-hot encode the labels, especially if your model will use categorical cross-entropy loss.

One-Hot Encoding converts categorical labels into binary vectors (one-hot encoded format). Each class label is represented as a binary vector with 1 for the correct class and 0 for others. This is necessary for training models that use categorical_crossentropy as the loss function, such as a neural network.

In [ ]:
# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot = encoder.fit_transform(y_train.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot = encoder.transform(y_test.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.

In [ ]:
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot shape: {y_train_one_hot.shape}, y_test_one_hot shape: {y_test_one_hot.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

y_train_one_hot shape: (55152, 15), y_test_one_hot shape: (13917, 15)


## Step 2: Build the 1D-CNN Model
Call our `build_1d_cnn_model` function to build our model

In [ ]:
# Determine the input shape for the model
input_shape = (X_train.shape[1], X_train.shape[2])

# Determine the number of output classes (num_classes)
num_classes = y_train_one_hot.shape[1]

# Build and compile the model
# The function will return a compiled model ready for training
model = build_1d_cnn_model(input_shape, num_classes)

c:\Users\seanc\miniconda3\envs\pdiot\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 48, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 22, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 11, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       180,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 208,207 (813.31 KB)

 Trainable params: 208,207 (813.31 KB)

 Non-trainable params: 0 (0.00 B)

## Step 3: Train the CNN Model

Train the 1D CNN model using the training data and validate on the test data. The model will learn to map input sliding windows to their corresponding activity labels.

`model.fit()` is used to train the neural network model. It takes several parameters:
* `X_train`: The input training data (sliding windows), with shape (num_samples, window_size, num_features).
* `y_train_one_hot`: The corresponding one-hot encoded labels for the training data, with shape (num_samples, num_classes).
* `epochs`: Number of times the entire training dataset is passed through the model. You can try adjusting the number of epochs and compare the difference in model performance. In this case, we are training for 20 epochs, meaning the model will see the entire training set 20 times.
* `batch_size`: Number of samples processed before the model's weights are updated. Here, the batch size is set to 32, meaning the model will process 32 samples at a time before updating its parameters.
* `validation_data`: This parameter allows us to evaluate the model's performance on the test data after each epoch.
*`(X_test, y_test_one_hot)`: These are the input test data and corresponding one-hot encoded test labels.

In [ ]:
history = model.fit(X_train, y_train_one_hot,
                    epochs=20,         # Train the model for 20 epochs
                    batch_size=32,     # Use a batch size of 32
                    validation_data=(X_test, y_test_one_hot))  # Validate on the test set after each epoch

Epoch 1/20
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.4134 - loss: 1.7623 - val_accuracy: 0.5598 - val_loss: 1.0690
Epoch 2/20
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.5379 - loss: 1.0960 - val_accuracy: 0.5797 - val_loss: 1.0249
Epoch 3/20
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.5852 - loss: 0.9821 - val_accuracy: 0.5914 - val_loss: 1.0226
Epoch 4/20
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6081 - loss: 0.9418 - val_accuracy: 0.6162 - val_loss: 0.9389
Epoch 5/20
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6325 - loss: 0.8591 - val_accuracy: 0.5995 - val_loss: 0.9835
Epoch 6/20
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6498 - loss: 0.8248 - val_accuracy: 0.6099 - val_loss: 0.9447
Epoch 7/20
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6575 - loss: 0.7906 - val_accuracy: 0.6184 - val_loss: 0.9598
Epoch 8/20
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6755 - loss: 0.7587 - 

`val_accuracy` is the accuracy of the model on the validation data (in this case X_test and y_test_one_hot). The `accuracy` is the training accuracy for the current epoch.

## Step 4: Evaluate the Model
After training, you can evaluate the model on the test set:

In [ ]:
# Get predicted probabilities for the test set
y_pred_probs = model.predict(X_test)

# Convert the predicted probabilities to class labels (taking the argmax of the probabilities)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Convert the true test labels from one-hot encoding back to class labels
y_true_classes = np.argmax(y_test_one_hot, axis=1)

# Generate the classification report
report = classification_report(y_true_classes, y_pred_classes, digits=4)

# Print the classification report
print(report)

435/435 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0     0.5389    0.6135    0.5738       282
           1     0.4433    0.3158    0.3689       285
           2     0.3282    0.4542    0.3811       568
           3     0.7451    0.8000    0.7716       285
           4     0.6303    0.6237    0.6270       287
           5     0.0204    0.0035    0.0060       284
           6     0.2031    0.1373    0.1639       284
           7     0.2786    0.3345    0.3040       284
           8     0.0309    0.0106    0.0157       284
           9     0.4414    0.6580    0.5284       269
          10     0.9091    0.8451    0.8759       284
          11     0.2826    0.2825    0.2826      1671
          12     0.6911    0.6948    0.6929      1658
          13     0.7935    0.4909    0.6065      1644
          14     0.7349    0.8371    0.7827      5548

    accuracy                         0.6149     13917
   macro avg     0.4714    0.4734    0.

As you can see from the model performance results, the classification performance isn't exactly impressive. For Coursework 3, your group should explore and experiment with various models, parameters, and techniques in order to improve your model's performance.

# Exporting your model to TFLite

You can use the TFLiteConverter class provided by TensorFlow to convert your trained model into the TensorFlow Lite format. We export models to TensorFlow Lite (TFLite) for several reasons, primarily because TFLite is designed for deployment on edge devices, such as mobile phones, embedded systems, IoT devices, and microcontrollers, where computational resources and power are limited. This is necessary as you will be running your ML models on your Android devices to perform live classification.

In [ ]:
# Convert the trained Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # model is your trained Keras model
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully exported to model.tflite")

Saved artifact at '/tmp/tmp4gyehid9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name='keras_tensor_74')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  132889844347856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132889845628992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132889845745440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132889845748432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132889845754768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132889845758288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132889845757584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132889845944336: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model successfully exported to model.tflite


# Good job!
This is the end of Lab 3. In the next lab, you will focus on deploying your machine learning model onto your Android App in order to classify activities in real-time.